In [1]:
from generatengrams import ngrammatch
from Contexts import *
#from spellcorrection import *
import json
from Intents import *
import random
import os
import re
import time
import sqlite3

#Spell check code

In [2]:
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('word_corpus.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(uInput): 
    "Most probable spelling correction for word."
    uInput_word_list = re.findall("[A-Z\-\']{2,}(?![a-z])|[A-Z\-\'][a-z\-\']+(?=[A-Z])|[\'\w\-]+",uInput)
    corrected_uinput_word_list = []
    for word in uInput_word_list:
        if len(word) > 2:
            corrected_uinput_word_list.append(max(candidates(word), key=P))
        else: 
            corrected_uinput_word_list.append(word)
    return " ".join(str(corrected_word) for corrected_word in corrected_uinput_word_list)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


In [3]:
cab_alterName =["cab", "taxi", "vehicle", "ride", "car", "shuttle"]
restaurant_alterName =["restaurant", "dine", "cuisine", "dining room", "bhojanalya", "Food court", "Coffee Shop"]

In [4]:
def bookRestaurant(attributes):
    conn = sqlite3.connect('./Database/restuarant.db')
    #print("Opened database successfully")
    #print()
    #"+attributes['cuisine']+"
    #print(attributes)
    cursor = conn.execute("SELECT * from RESTAURANTMASTER where CUISINETYPE ='"+attributes['cuisine'].upper()+"' and COSTTYPE = '"+attributes['type'].upper()+"'")
    #print(cursor)
    msg = '';  
    msg +='Your search is successful'
    count=0    
    for row in cursor:
        count += 1
        msg+= '\n Proposed restaurent name:'+row[2]+' ,with cuisine type: '+row[3]+' , budget type:'+row[4]+' ,location: '+attributes['region']
        break
        
    if(count==0):
        msg = ''; 
        cursor = conn.execute("SELECT * from RESTAURANTMASTER where CUISINETYPE like'"+attributes['cuisine'].upper()+"' or COSTTYPE = '"+attributes['type'].upper()+"'")
        for row in cursor:
            msg +='we could not search exact match, but following is our recommendation:: \n' 
            msg+= '\n Proposed restaurent name:'+row[2]+' ,with cuisine type: '+row[3]+' , budget type:'+row[4]+' ,location: '+attributes['region']
            break
    conn.close() 
    
    return msg

def bookCab(attributes):
    conn = sqlite3.connect('./Database/cabbooking.db')
    #print(attributes)
    cursor = conn.execute("SELECT * from CABMASTER where LUGGAGEBAGS <="+attributes['luggage']+" or NOOFPEOPLE <= "+attributes['people']+" ORDER BY NOOFPEOPLE DESC")
    msg = '';
    for row in cursor:
        msg+= 'Your booking has been accepted , below are your cab details:\ncar no: '+row[1]+', with car model: '+row[3]+', Driver name: '+row[2]+' is booked'
        break
    #print("Operation done successfully")
    conn.close()
    #print("Opened database successfully"+msg)
    return msg

def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent
    as mentioned in the intent config file'''
    '''Performs actions pertaining to current intent
    for action in current_intent.actions:
        if action.contexts_satisfied(active_contexts):
            return perform_action()
    '''
    context = IntentComplete()
    #print(current_intent)
    #print(attributes)
    #print(context)
    msg = ''
    if(attributes and ('cuisine' in attributes)):
        #print("restaurant")
        msg = bookRestaurant(attributes)
    elif(attributes and ('destination' in attributes)):
        #print("cab")
        msg = bookCab(attributes)
        
    exit_message = '\n\nYou can continue with another "cab booking"/"restaurant search" or type "exit" to close the chat'
    return msg + exit_message , context

def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    para_name =  None
    
    if(current_intent == None):
        intent_wait = "I can only help you with either a 'Cab booking' or a 'Restaurant search'. Please choose from these options only"
        return intent_wait, context, para_name
    
    for para in current_intent.params:
        if para.required:
            if para.name not in attributes:
                para_name = para.name
                return random.choice(para.prompts), context, para_name      
    return None, context, para_name


def input_processor(user_input, context, attributes, intent, para_name):
    '''Spellcheck and entity extraction functions go here'''
    
    #uinput = TextBlob(user_input).correct().string
    
    #update the attributes, abstract over the entities in user input
    attributes, cleaned_input = getattributes(user_input, context, attributes, para_name)
    return attributes, cleaned_input

def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        #print(intent)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    scores = ngrammatch(clean_input)
    scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])
    #print(scores)
    #print 'scores', scores
    

    if(current_intent==None):
        for i in range(len(cab_alterName)):
            if(cab_alterName[i] in clean_input):
                return loadIntent('params/params.cfg','bookcab')
        for i in range(len(restaurant_alterName)):
            if(restaurant_alterName[i] in clean_input):
                return loadIntent('params/params.cfg', 'searchrestaurant')
        else:
            #return 
            return loadIntent('params/params.cfg',scores[-1][0])
    else:
        #print 'same intent'
        return current_intent

def getattributes(uinput,context,attributes, para_name):
    '''This function marks the entities in user input, and updates
    the attribut`es dictionary'''
    #Can use context to context specific attribute fetching
    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:

        files = os.listdir('./entities/')
        entities = {}
        # read all the .dat files for entities list creation
        for fil in files:
            lines = open('./entities/'+fil).readlines()
            for i, line in enumerate(lines):
                lines[i] = line[:-1]
            entities[fil[:-4]] = '|'.join(lines) # -4 removes .dat extension
        #print(entities)
        # entity holds value like destination, luggage, people etc.
        for entity in entities:
            for i in entities[entity].split('|'):
                if (uinput.lower() in i.lower()) and ((para_name == entity) or (para_name == None)) :
                    attributes[entity] = i
        for entity in entities:
                uinput = re.sub(entities[entity],r'$'+entity,uinput,flags=re.IGNORECASE)
        # here uinput changes to $entity        
        if context.name=='GetRegNo'  and context.active:
            match = re.search('[0-9]+', uinput)
            if match:
                uinput = re.sub('[0-9]+', '$regno', uinput)
                attributes['RegNo'] = match.group()
                context.active = False

        return attributes, uinput

def cab_recommandation():
    recommendation = "Mahindra verito"
    return recommendation

In [5]:
class Session:
    def __init__(self, attributes= None, active_contexts=[FirstGreeting(),IntentComplete]):
        
        '''Initialise a default session'''
        #Contexts are flags which control dialogue flow
        self.active_contexts = active_contexts
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
        self.para_name = None
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()

    def reply(self, user_input):
        '''Generate response to user input'''
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent, self.para_name)
        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        
        prompt, self.context, self.para_name = check_required_params(self.current_intent, self.attributes, self.context)

        #prompt being None means all parameters satisfied, perform the intent action
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete': 
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
 
        return prompt

In [7]:
session = Session()

print ('BOT: Hi! How may I assist you?')
time.sleep(0.5)
while True:
    
    inp1 = input('User: ')
    inp = correction(inp1.lower())
    #print(inp)

    if(inp == "exit"):
        print("Thanks. Have a nice day")

        break
    print ('BOT:', session.reply(inp))

BOT: Hi! How may I assist you?
User: I want cab
BOT: What's your destination location?
User: kondapur
BOT: How many pieces of luggages you have?: Type between 0 to 10
User: 3
BOT: For how many people the cab is required?: Type between 1 to 10
User: 4
BOT: Do you want an AC or a Non-AC cab?
User: AC
BOT: Your booking has been accepted , below are your cab details:
car no: 2222, with car model: MARUTI SWIFT, Driver name: Max is booked

You can continue with another "cab booking"/"restaurant search" or type "exit" to close the chat
User: exit
Thanks. Have a nice day
